In [ ]:
import pickle as pickle
import numpy as np
base_path="../data/data_m3/"
with open(base_path+'binary_train_codes_x.pkl', 'rb') as f0:
  binary_train_codes_x = pickle.load(f0)

with open(base_path+'binary_test_codes_x.pkl', 'rb') as f1:
  binary_test_codes_x = pickle.load(f1)

train_codes_y = np.load(base_path+'train_codes_y.npy')
train_visit_lens = np.load(base_path+'train_visit_lens.npy')

test_codes_y = np.load(base_path+'test_codes_y.npy')
test_visit_lens = np.load(base_path+'test_visit_lens.npy')
train_pids = np.load(base_path+'train_pids.npy')

test_pids = np.load(base_path+'test_pids.npy')
with open(base_path+'patient_time_duration_encoded.pkl', 'rb') as f80:
  patient_time_duration_encoded = pickle.load(f80)

In [ ]:
import numpy as np
import pickle

with open(base_path+"code_map.pkl", "rb") as f:
    code_map = pickle.load(f)

inv_code_map = {v-1: k for k, v in code_map.items()}
print(code_map)
def fun1(idx):
    result=[]
    for id in idx:
        result.append(inv_code_map[id])
    return result

In [ ]:
import pandas as pd
df = pd.read_csv(base_path+'diagnosis_icd9_ontology.csv')
code2name = dict(zip(df['code'], df['name']))

In [ ]:
def get_diag_str(diag_list):
    temp_str_list=[]
    for diag in diag_list:
        temp_str_list.append(f'"{code2name[diag]}"')
    temp_str=', '.join(temp_str_list)
    return temp_str

In [ ]:
import inflect

def number_to_capitalized_ordinal(n):
    p = inflect.engine()
    return p.ordinal(p.number_to_words(n)).capitalize()

print(number_to_capitalized_ordinal(1)) 
print(number_to_capitalized_ordinal(42)) 


In [ ]:
results=[]
ode_idx=0
top_k=50
for i in range(len(test_codes_y)):
    if test_visit_lens[i]<2:
        continue
    pids=test_pids[i]
    hist_diag=binary_test_codes_x[i]
    target_diag=test_codes_y[i]
    hist_diag_code=[]
    all_hist_diag_code=[]
    ode_idx+=1

    for v in hist_diag:
        hist_diag_code.append(fun1(np.where(v == 1)[0]))
        all_hist_diag_code.extend(hist_diag_code[-1])
    all_hist_diag_code=list(set(all_hist_diag_code))
    instruction="You are an experienced medical diagnosis expert specializing in longitudinal disease progression and comorbidity reasoning. Your goal is to infer the most plausible diseases for the patient’s final visit based solely on their historical diagnoses and the clinical logic of disease progression, recurrence, and complications."

    duration=patient_time_duration_encoded[pids]
    
    input_=""
    for idx,code_list in enumerate(hist_diag_code):
        if idx==0:
            cur_duration=0
            temp=number_to_capitalized_ordinal(idx+1)+f" Visit:"
        else:
            cur_duration+=duration[idx]
            temp=number_to_capitalized_ordinal(idx+1)+f" Visit: ({cur_duration} days later):"

        temp+=f"\n- Diagnoses："+"{"+get_diag_str(code_list)+"}"
        input_+=temp+"\n\n"
    temp=f"Final Visit ({sum(duration)} days later):\n\n"
    temp+=f"Your task:\n- Please infer the likely diseases at this visit by integrating longitudinal progression, complication chains, and chronic recurrence patterns. Prioritize diseases that are progressive, recurrent, or secondary to prior pathologies. De-emphasize transient, resolved, or treatment-related acute conditions unlikely to persist."+"\nDirectly provide the reordered list of disease names in descending order of likelihood. \nOutput format:\nAnswer: <Disease 1>, <Disease 2>, <Disease 3>, ..."
    input_+=temp+"\n"
    result={
        "pid":pids,
        "hist_diag_code":hist_diag_code,
        "target_diag_code":fun1(np.where(target_diag == 1)[0]),
        "instruction":instruction,
        "input":input_
        
    }

    results.append(result)

In [ ]:
print(results[0]['instruction'] + '\n' + results[0]['input'])


In [ ]:
print(results[0]['instruction'] + '\n' + results[0]['input'])


In [ ]:
len(results)

In [ ]:
import json
def np_encoder(obj):
    if isinstance(obj, (np.integer, np.floating, np.bool_)):
        return obj.item()          
    if isinstance(obj, np.ndarray):
        return obj.tolist()       
    return str(obj)            

with open(f"prompt_data/m3_cra_r1.json", "w",
          encoding="utf-8") as f:
    json.dump(results, f, ensure_ascii=False,default=np_encoder, indent=2)